In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from util import getData
%matplotlib inline

##### ハイパーパラメタの宣言

In [2]:
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils
from keras import backend as K

batch_size = 128
nb_classes = 7
nb_epoch = 12

img_rows, img_cols = 48, 48
nb_filters = 20
pool_size = (2, 2)
pool_stride_size = (2, 2)
kernel_size = (5, 5)

Using TensorFlow backend.


##### データの読み込み

In [3]:
X_train, X_test, y_train, y_test = getData()

In [4]:
X_train.shape

(32633, 2304)

データが既にフラットになっているので、これを4次元データに変換する。(batch size, color, width, height)

In [5]:
## Caution!! ##
# kerasはバックエンドでtheanoもしくはtensorflowを動かしています。
# tensorflowとtheanoでinputとして渡すデータの形式が若干異なります。
# そのため、以下のように、それぞれのフレームワークごとにデータを整形し直します。
if K.image_dim_ordering() == 'th': # theanoがバックエンドで動くように設定されている場合
    X_train = X_train.reshape(X_train.shape[0], 1, img_rows, img_cols)
    X_test = X_test.reshape(X_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:# tensorflowがバックエンドで動くように設定されている場合
    X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
    X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

In [6]:
X_train.shape

(32633, 48, 48, 1)

4次元になりました。

In [7]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

X_train shape: (32633, 48, 48, 1)
32633 train samples
4093 test samples


##### yのone-hot encoding

In [8]:
from sklearn.preprocessing import LabelBinarizer, MultiLabelBinarizer
lb = LabelBinarizer()
y_train = lb.fit_transform(y_train)
y_test = lb.fit_transform(y_test)

##### CNN modelの作成

In [9]:
model = Sequential()

model.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1], border_mode='same', dim_ordering=K.image_dim_ordering(), input_shape=input_shape))
model.add(Activation('tanh'))
model.add(MaxPooling2D(pool_size=pool_size, strides=pool_stride_size, dim_ordering=K.image_dim_ordering()))
model.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1], border_mode='same', dim_ordering=K.image_dim_ordering()))
model.add(Activation('tanh'))
model.add(MaxPooling2D(pool_size=pool_size, strides=pool_stride_size, dim_ordering=K.image_dim_ordering()))
model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(500))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(300))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))

In [10]:
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [11]:
model.fit(X_train, y_train, batch_size=batch_size, nb_epoch=nb_epoch,
          verbose=1, validation_data=(X_test, y_test))

Train on 32633 samples, validate on 4093 samples
Epoch 1/12
32633/32633 [==============================] - 131s - loss: 1.9087 - acc: 0.2223 - val_loss: 1.8395 - val_acc: 0.2688
Epoch 2/12
32633/32633 [==============================] - 139s - loss: 1.7807 - acc: 0.3091 - val_loss: 1.6752 - val_acc: 0.3667
Epoch 3/12
32633/32633 [==============================] - 133s - loss: 1.6639 - acc: 0.3682 - val_loss: 1.5823 - val_acc: 0.4210
Epoch 4/12
32633/32633 [==============================] - 139s - loss: 1.5562 - acc: 0.4141 - val_loss: 1.5524 - val_acc: 0.4310
Epoch 5/12
32633/32633 [==============================] - 136s - loss: 1.4740 - acc: 0.4442 - val_loss: 1.4886 - val_acc: 0.4425
Epoch 6/12
32633/32633 [==============================] - 149s - loss: 1.4069 - acc: 0.4673 - val_loss: 1.4987 - val_acc: 0.4571
Epoch 7/12
32633/32633 [==============================] - 136s - loss: 1.3539 - acc: 0.4825 - val_loss: 1.5134 - val_acc: 0.4625
Epoch 8/12
32633/32633 [========================

手元の環境accuracy0.52あたりで収束